<a href="https://colab.research.google.com/github/innocentmatutu/Machine-learning/blob/main/Imputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import zipfile
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

with zipfile.ZipFile('/content/archive (4).zip') as z:
  with z.open('weather_dirty.csv') as f:
    df = pd.read_csv(f , sep=';')

df['Date'] = pd.to_datetime(df['Date'],format='mixed')
df = df.dropna(axis=1, how='all')
df = df.sort_values('Date').reset_index(drop=True)

numeric_cols = ['Temperature_day', 'Temperature_night', 'AirPressure_hPa',
                'WindSpeed_km/h', 'Precipitation_mm']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

for col in df.select_dtypes(include=['number']).columns:
  #df[col].dropna(inplace=True)
  #df[col].fillna(df[col].mean(),inplace=True)
  #df[col].fillna(df[col].mode(),inplace=True)
  df[col].fillna(df[col].median(),inplace=True)

for col in df.select_dtypes(include=['object']).columns:
  le = LabelEncoder()
  df[col] = le.fit_transform(df[col])


features=['Temperature_day','Temperature_night','AirPressure_hPa','WindSpeed_km/h','Precipitation_mm','Wind_direction']
X=df[features]
y=df['Outlook']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)
rf=RandomForestClassifier(n_estimators=100,random_state=1)
rf.fit(X_train,y_train)
predictions=rf.predict(X_valid)

print(f"Accuracy score: {accuracy_score(y_valid,predictions)}")
print(f"Precison score: {precision_score(y_valid,predictions, average='weighted')}")
print(f"Recall score: {recall_score(y_valid,predictions,average='weighted')}")
print(f"F1 score: {f1_score(y_valid,predictions,average='weighted')}")


/tmp/ipython-input-3102890118.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(),inplace=True)


Accuracy score: 0.7012987012987013
Precison score: 0.6176194618778351
Recall score: 0.7012987012987013
F1 score: 0.6456899475332655


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import zipfile
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

with zipfile.ZipFile('/content/archive (4).zip') as z:
  with z.open('weather_dirty.csv') as f:
    df = pd.read_csv(f , sep=';')

df = df.dropna(axis=1, how='all')
for col in df.select_dtypes(include=['object']).columns:
  le = LabelEncoder()
  df[col] = le.fit_transform(df[col])

features = ['Temperature_day', 'Temperature_night', 'AirPressure_hPa', 'WindSpeed_km/h', 'Precipitation_mm','Wind_direction']

X=df[features]
y=df['Outlook']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)



def impute_with_extension(X_train, X_valid):
    cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
    X_train_plus = X_train.copy()
    X_valid_plus = X_valid.copy()

    for col in cols_with_missing:
        X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
        X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

    imputer = SimpleImputer(strategy='mean')
    imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train_plus), columns=X_train_plus.columns)
    imputed_X_valid = pd.DataFrame(imputer.transform(X_valid_plus), columns=X_valid_plus.columns)

    return imputed_X_train, imputed_X_valid

imputed_X_train, imputed_X_valid = impute_with_extension(X_train, X_valid)
rf=RandomForestClassifier(n_estimators=100,random_state=1)
rf.fit(imputed_X_train,y_train)
predictions=rf.predict(imputed_X_valid)

print(f"Accuracy score: {accuracy_score(y_valid,predictions):4f}")
print(f"Precison score: {precision_score(y_valid,predictions, average='weighted'):.4f}")
print(f"Recall score: {recall_score(y_valid,predictions,average='weighted'):.4f}")
print(f"F1 score: {f1_score(y_valid,predictions,average='weighted'):.4f}")




Accuracy score: 0.779221
Precison score: 0.7713
Recall score: 0.7792
F1 score: 0.7665


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
